In [ ]:
!pip install unidecode

In [ ]:
# Imports necessários
import pandas as pd
import numpy as np
import re
from textblob import TextBlob
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from wordcloud import WordCloud, STOPWORDS
from PIL import Image
from collections import Counter
from unidecode import unidecode

In [ ]:
# Limpeza de texto
stopwords = set(STOPWORDS)
stopwords.update(["palavras para remover"])

# Combine todos os comentários em uma única string
text = ' '.join(df['comentario'])

# Crie a nuvem de palavras
wordcloud = WordCloud(stopwords=stopwords, background_color='white',
                      width=800, height=600, max_words=200).generate(text)

# Visualize a nuvem de palavras
plt.figure(figsize=(10, 8))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Word Cloud dos Comentários')
plt.show()

In [ ]:
# Suponhamos que 'df' seja o seu DataFrame e 'comentario' a coluna com os comentários
filtered_df = df[df['comentario'].str.contains(r'\b(não|Não)\b', case=False, regex=True)]

# Exibindo as frases completas filtradas
for comment in filtered_df['comentario']:
    print(comment)



In [ ]:
# Combine todos os comentários em uma única string
text = ' '.join(df['comentario'])

# Lista personalizada de stopwords
custom_stopwords = (["palavras para remover"])

# Divida a string em palavras e remova as stopwords
words = [word for word in text.split() if word.lower() not in custom_stopwords]

# Conte a frequência de cada palavra após remover as stopwords
word_freq = Counter(words)

# Converta o dicionário de frequência em um DataFrame para facilitar a visualização
word_freq_df = pd.DataFrame(list(word_freq.items()), columns=['Palavra', 'Frequência'])

# Ordene o DataFrame pela frequência em ordem decrescente
word_freq_df = word_freq_df.sort_values(by='Frequência', ascending=False)

# Exiba as 10 palavras mais frequentes
print(word_freq_df.head(10))

# Visualize as 10 palavras mais frequentes em um gráfico de barras
plt.figure(figsize=(10, 6))
plt.bar(word_freq_df['Palavra'][:10], word_freq_df['Frequência'][:10], color='blue')
plt.xlabel('Palavra')
plt.ylabel('Frequência')
plt.title('Top 10 Palavras Mais Frequentes (Sem Stopwords)')
plt.xticks(rotation=45)
plt.show()

In [ ]:
def remove_accents(input_str):
    return unidecode(input_str)

def classify_comment(comment):
    negative_keywords = ["pessimo", "ruim", "erro", "bug", "reclamacao", "reclamam", "lento", "dificil", "melhorar", "erro", "aperfeicoar", "dificil", "melhoras", "necessita", "dificultou", "faltam", "não"] #palavras que você deseja mapear como negativas
    positive_keywords = ["bom", "otimo", "eficiente", "boa", "excelente", "gosto", "flexibilidade", "acesso", "impecavel", "pratico", "ideal", "facil", "eficaz", "atenciosa", "praticidade","rapido"] #palavras que você deseja mapear como positivas


    # Remover acentuação das palavras no comentário
    words = [remove_accents(word.lower()) for word in comment.split()]

    negative_count = sum(word in negative_keywords for word in words)
    positive_count = sum(word in positive_keywords for word in words)

    if negative_count > positive_count:
        return 'Negativo'
    elif negative_count < positive_count:
        return 'Positivo'
    else:
        return 'Neutro'

# Aplicar a função de classificação a cada comentário na coluna 'comentario' do DataFrame
df['classificacao'] = df['comentario'].apply(classify_comment)

# Mostrar os resultados
print(df[['comentario', 'classificacao']])


In [ ]:
# Contagem de cada classe
class_counts = df['classificacao'].value_counts()

# Criando o gráfico
plt.figure(figsize=(8, 6))
class_counts.plot(kind='bar', color=['#07AA8B', '#811AC0', '#CC4C1F'])

# Título e rótulos dos eixos
plt.title('Distribuição de Classificações de Comentários')
plt.xlabel('Classificação')
plt.ylabel('Número de Comentários')

# Mostrar o gráfico
plt.show()

In [ ]:
def remove_accents(input_str):
    return unidecode(input_str)

def classify_comment(comment):
    negative_keywords = ["preencher com palavras negativas como acima"]
    positive_keywords = ["preencher com palavras positivas como acima"]

    words = [remove_accents(word.lower()) for word in comment.split()]

    negative_count = sum(word in negative_keywords for word in words)
    positive_count = sum(word in positive_keywords for word in words)

    if negative_count > positive_count:
        return 'Negativo'
    elif negative_count < positive_count:
        return 'Positivo'
    else:
        return 'Neutro'

df['classificacao'] = df['comentario'].apply(classify_comment)

def classify_nps(n):
    if n >= 9:
        return 'Promotor'
    elif n <= 6:
        return 'Detrator'
    else:
        return 'Neutro'

df['Classificação NPS'] = df['nota'].apply(classify_nps)

counts = df.groupby('Classificação NPS')['classificacao'].value_counts().unstack(fill_value=0)
normalized_counts = counts.divide(counts.sum(axis=1), axis=0) * 100

# Definir cores personalizadas
cores = {'Negativo': '#CC4C1F', 'Neutro': '#07AA8B', 'Positivo': '#811AC0'}

# Plotar o gráfico de barras empilhadas com cores personalizadas
ax = normalized_counts.plot(kind='bar', stacked=True, figsize=(10, 6), color=[cores[col] for col in normalized_counts.columns])

plt.title('Distribuição de Sentimentos por Categoria NPS (em %)')
plt.xlabel('Categoria NPS')
plt.ylabel('Porcentagem de Comentários')
plt.xticks(rotation=0)
plt.legend(title='Sentimento')

# Adicionar porcentagens dentro das barras
for p in ax.patches:
    width, height = p.get_width(), p.get_height()
    x, y = p.get_xy()
    ax.text(x + width/2, y + height/2, f'{height:.1f}%', ha='center', va='center')


plt.show()